In [2]:
import re
import pickle
import random

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

In [49]:
# read source file
reviews_df = pd.read_csv("Resources/all_reviews_data.csv", encoding='ISO-8859-1')


# count of movie plot summaries
# len(corpus)
reviews_df

,Review_ID,Rating,Reviewer_Location,Review_Text,Branch,Year,Month,is_good_review,review_clean,neg,...,word_yet,word_york,word_you,word_young,word_yr,word_yrs,word_yum,word_yummy,word_zero,word_zone
0,122944085,5,United States,Spotlessly clean and so organized considering ...,Disneyland_California,2011,10,1,spotlessly clean organize consider thousand vi...,0.000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,290851203,5,United Kingdom,"Pricey but worth it, unfortunately the big thu...",Disneyland_Paris,2015,7,1,pricey worth unfortunately big thunder mountai...,0.115,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,168087311,4,Canada,"Disneyland is a magical place. Everyone, parti...",Disneyland_California,2013,7,1,disneyland magical place everyone particularly...,0.000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,282417360,5,Pakistan,Disney land is a must when visiting Paris. The...,Disneyland_Paris,2015,6,1,disney land must visit paris frozen show best ...,0.028,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,160953212,3,India,"The HK Disneyland is definitely small, but it'...",Disneyland_HongKong,2012,9,0,hk disneyland definitely small definitely fun ...,0.000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3999,290217727,5,Australia,There's a reason why they call Disneyland The...,Disneyland_California,2015,7,1,there's reason call disneyland happy place ear...,0.042,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4000,174264188,1,United Kingdom,I visited Disneyland Paris with my family (tee...,Disneyland_Paris,2013,8,0,visit disneyland paris family teenage daughter...,0.081,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4001,638583788,5,United States,What a great day we had we did so much before ...,Disneyland_California,2018,12,1,great day much great guide grandson amaze well...,0.000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4002,226159775,5,Philippines,It has been a childhood dream of mine to go to...,Disneyland_California,2013,10,1,childhood dream mine go disneyland whether you...,0.000,...,0.0,0.0,0.0,0.432261,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
corpus = reviews_df["Review_Text"]
corpus

0       Spotlessly clean and so organized considering ...
1       Pricey but worth it, unfortunately the big thu...
2       Disneyland is a magical place. Everyone, parti...
3       Disney land is a must when visiting Paris. The...
4       The HK Disneyland is definitely small, but it'...
                              ...                        
3999    There's a reason why they call Disneyland  The...
4000    I visited Disneyland Paris with my family (tee...
4001    What a great day we had we did so much before ...
4002    It has been a childhood dream of mine to go to...
4003    Well my wife and I visted in October 2012 for ...
Name: Review_Text, Length: 4004, dtype: object

In [42]:
# clean text
reviews = [re.sub("[^a-z' ]", "", i) for i in corpus]
reviews

['potlessly clean and so organized considering the thousands of visitors each day',
 'ricey but worth it unfortunately the big thunder mountain ride broke down many times ast pass needs to be used and helps cut down waits hugely lso best to go on rides in the evening during or after the parade he dreams show is absolutely amazing and not to be missed y kids aged  and  loved these rides the best  big thunder mountain tower of terror ratatouille  out of the world disney special effects pirates of the aribbean star tours eter an ',
 'isneyland is a magical place veryone particularly those with kids knows thishe park was great for our familyhe edi raining cademy and tar ours were the favoriteslso really enjoyed the atterhorn irates ride and plash ountainid find the park seemed a little old in some of the venuest is isney thoughso you still have to love it ',
 'isney land is a must when visiting aris he rozen show was the best thing  had ll the rides are great ig hunder mountain got faulty 

In [43]:
# create sequences of length 5 tokens
def create_seq(text, seq_len = 5):
    
    sequences = []

        # if the number of tokens in 'text' is greater than 5
    if len(text.split()) > seq_len:
        for i in range(seq_len, len(text.split())):
        # select sequence of tokens
           seq = text.split()[i-seq_len:i+1]
        # add to the list
           sequences.append(" ".join(seq))
        return sequences

        # if the number of tokens in 'text' is less than or equal to 5
    else:
      
     return [text]

In [50]:
seqs = [create_seq(i) for i in corpus]

# merge list-of-lists into a single list
seqs = sum(seqs, [])

# count of sequences
len(seqs)

485287

In [101]:
# create inputs and targets (x and y)
x = []
y = []


for s in seqs:
    x.append(" ".join(s.split()[:-1]))
    y.append(" ".join(s.split()[1:]))
    


In [102]:
# create integer-to-token mapping
int2token = {}
cnt = 0

for w in set(" ".join(corpus).split()):
    int2token[cnt] = w
    cnt+= 1

# create token-to-integer mapping
token2int = {t: i for i, t in int2token.items()}

token2int["the"], int2token[14271]

(11904, 'shameful.')

In [103]:
# set vocabulary size
vocab_size = len(int2token)
vocab_size

32053

In [104]:
def get_integer_seq(seq):
    return [token2int[w] for w in seq.split()]

# convert text sequences to integer sequences
x_int = [get_integer_seq(i) for i in x]
y_int = [get_integer_seq(i) for i in y]

# convert lists to numpy arrays
x_int = np.array(x_int)
y_int = np.array(y_int)

C:\Users\ajrfl\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':
C:\Users\ajrfl\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # Remove the CWD from sys.path while we load stuff.


In [105]:
def get_batches(arr_x, arr_y, batch_size):
    # iterate through the arrays
    prv = 0
    for n in range(batch_size, arr_x.shape[0], batch_size):
        print(arr_x)
        x = arr_x[prv:n,:]
        y = arr_y[prv:n,:]
        prv = n
    yield x, y

In [106]:
def init_hidden(self, batch_size):
        ''' initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        # if GPU is available
        if (torch.cuda.is_available()):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                    weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        
        # if GPU is not available
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                    weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [107]:
class WordLSTM(nn.Module):
    
    def __init__(self, n_hidden=256, n_layers=4, drop_prob=0.3, lr=0.001):
        super().__init__()

        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        self.emb_layer = nn.Embedding(vocab_size, 200)

        ## define the LSTM
        self.lstm = nn.LSTM(200, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## define the fully-connected layer
        self.fc = nn.Linear(n_hidden, vocab_size)      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''

        ## pass input through embedding layer
        embedded = self.emb_layer(x)     
        
        ## Get the outputs and the new hidden state from the lstm
        lstm_output, hidden = self.lstm(embedded, hidden)
        
        ## pass through a dropout layer
        out = self.dropout(lstm_output)
        
        #out = out.contiguous().view(-1, self.n_hidden) 
        out = out.reshape(-1, self.n_hidden) 

        ## put "out" through the fully-connected layer
        out = self.fc(out)

        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        # if GPU is available
        if (torch.cuda.is_available()):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                    weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        
        # if GPU is not available
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                    weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [108]:
# instantiate the model
net = WordLSTM()

print(net)

WordLSTM(
  (emb_layer): Embedding(32053, 200)
  (lstm): LSTM(200, 256, num_layers=4, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=32053, bias=True)
)


In [109]:
def train(net, epochs=10, batch_size=32, lr=0.001, clip=1, print_every=32):
    
    # optimizer
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    
    # loss
    criterion = nn.CrossEntropyLoss()
    
    # push model to GPU
#     net.cuda()
    
    counter = 0

    net.train()

    for e in range(epochs):

        # initialize hidden state
        h = net.init_hidden(batch_size)
        print(x_int, y_int)
        for x, y in get_batches(x_int, y_int, batch_size):
            counter+= 1
            
            # convert numpy arrays to PyTorch arrays
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            # push tensors to GPU
            inputs, targets = inputs.cuda(), targets.cuda()

            # detach hidden states
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(-1))

            # back-propagate error
            loss.backward()

            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)

            # update weigths
            opt.step()            
            
            if counter % print_every == 0:
            
              print("Epoch: {}/{}...".format(e+1, epochs),
                    "Step: {}...".format(counter))

In [110]:
# train the model
train(net, batch_size = 32, epochs=20, print_every=256)

[list([22725, 6104, 11121, 28557, 5015])
 list([6104, 11121, 28557, 5015, 480])
 list([11121, 28557, 5015, 480, 11904]) ...
 list([18166, 341, 31563, 24060, 26336])
 list([341, 31563, 24060, 26336, 18218])
 list([31563, 24060, 26336, 18218, 8254])] [list([6104, 11121, 28557, 5015, 480])
 list([11121, 28557, 5015, 480, 11904])
 list([28557, 5015, 480, 11904, 2311]) ...
 list([341, 31563, 24060, 26336, 18218])
 list([31563, 24060, 26336, 18218, 8254])
 list([24060, 26336, 18218, 8254, 20545])]
[list([22725, 6104, 11121, 28557, 5015])
 list([6104, 11121, 28557, 5015, 480])
 list([11121, 28557, 5015, 480, 11904]) ...
 list([18166, 341, 31563, 24060, 26336])
 list([341, 31563, 24060, 26336, 18218])
 list([31563, 24060, 26336, 18218, 8254])]


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [19]:
# predict next token
def predict(net, tkn, h=None):
         
  # tensor inputs
  x = np.array([[token2int[tkn]]])
  inputs = torch.from_numpy(x)

In [20]:
  # detach hidden state from history
  h = tuple([each.data for each in h])

  # get the output of the model
  out, h = net(inputs, h)

  # get the token probabilities
  p = F.softmax(out, dim=1).data

  p = p.cpu()

  p = p.numpy()
  p = p.reshape(p.shape[1],)

  # get indices of top 3 values
  top_n_idx = p.argsort()[-3:][::-1]

    # randomly select one of the three indices
  sampled_token_index = top_n_idx[random.sample([0,1,2],1)[0]]

  # return the encoded value of the predicted char and the hidden state
  return int2token[sampled_token_index], h

  # function to generate text
def sample(net, size, prime='it is'):
        
    # # push to GPU
    # net.cuda()
    
    net.eval()

    # batch size is 1
    h = net.init_hidden(1)

    toks = prime.split()

    # predict next token
    for t in prime.split():
      token, h = predict(net, t, h)
    
    toks.append(token)

    return ' '.join(toks)

NameError: name 'h' is not defined

In [ ]:
#sample with default prime
sample(net, 15)

In [ ]:
#sample with different prime
sample(net, 15, prime = "one of the")
